In [1]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))

import ModelFramework as mf
import FwtModels.RectWing as rw

## Create Method to Calculate the Eigen Response of a Given model

Model defined by number of b and t shapes and model ID

Using settings for JEC Wing

In [2]:
def Model_Eigen(model_num,b_modes,t_modes):
    p = rw.base_params(b_modes + t_modes + 1)
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    
    dataset_name = 'JEC'
    # Set JEC Specific parameters
    p.rho.value = 1.225
    p.s_t.value = 1.345
    p.c.value = 0.15
    p.EI.value = 61
    p.GJ.value = 200
    p.rho_t.value = 10
    p.e_0.value = 0
    p.e_1.value = 0
    p.ratio_fwt.value = 0.2
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(0,40,81))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    #vars_ls.append((p.ratio_fwt,[0,0.1,0.2,0.3]))
    #vars_ls.append((p.ratio_DL,[0,0.05,0.1,0.2]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))

    variables = [k for k,v in vars_ls]
    
    calc_fixed = True if np.isin(model_num,np.array([1,2,3,4,5])) else False
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed,jac = False)   
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{model_num}_{dataset_name}.pkl')

In [15]:
b_modes = 3
t_modes = 3
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

#for k in range(8):
for k in [5,7]:
    pool.apply_async(Model_Eigen,args=(k,b_modes,t_modes))
pool.close()
pool.join()

Process ForkPoolWorker-33:
Process ForkPoolWorker-34:
Process ForkPoolWorker-38:
Process ForkPoolWorker-40:
Process ForkPoolWorker-39:
Process ForkPoolWorker-37:
Process ForkPoolWorker-36:
Process ForkPoolWorker-35:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap

  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/core/basic.py", line 1207, in _xreplace
    a_xr = _xreplace(rule)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/core/basic.py", line 1207, in _xreplace
    a_xr = _xreplace(rule)
  File "<ipython-input-14-093d506e9a10>", line 28, in Model_Eigen
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed)
  File "/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/sympy/core/basic.py", line 1207, in _xreplace
    a_xr = _xreplace(rule)
  File "/Users/fintan/Documents/GitHub/FwtModels/Workbooks/9_RectWingModelGen/../../FwtModels/RectWing/eigen_perm_params.py", line 25, in eigen_perm_params
    model_lin = model_mini.lin

KeyboardInterrupt: 

In [3]:
%time Model_Eigen(5,3,3)

/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


CPU times: user 1min 31s, sys: 247 ms, total: 1min 31s
Wall time: 1min 31s


In [4]:
t = pd.read_pickle(f'Eigen_3B3T-M5_JEC2.pkl')

In [20]:
t.head()

,Real,Imag,Frequency,Damping,Stable,Eigen Vector,Mode,Lambda,V,alpha_r,delta_m,q
0,0.000000e+00,7.396121,1.177129,6.123234e-17,True,[(6.240372270402153e-31+1.4118369479243174e-19...,0,0.174533,0.0,0.0,0.5,"[-0.08727804221311625, 0.04866805327869114, -0..."
1,1.281943e-14,15.781589,2.511718,9.494108e-16,True,[(-2.2211618967688533e-16+0.05757834495299774j...,1,0.174533,0.0,0.0,0.5,"[-0.08727804221311625, 0.04866805327869114, -0..."
2,-3.681244e-13,103.929036,16.540820,-3.491481e-15,True,[(1.1028822368375594e-17-0.004601831860951751j...,2,0.174533,0.0,0.0,0.5,"[-0.08727804221311625, 0.04866805327869114, -0..."
3,4.898183e-12,370.062095,58.897212,1.338391e-14,True,[(-1.6053868851729373e-17+0.000751251409476966...,3,0.174533,0.0,0.0,0.5,"[-0.08727804221311625, 0.04866805327869114, -0..."
4,-1.809306e-11,486.391549,77.411619,-3.724226e-14,True,[(-2.974211605205383e-17+0.0004234529989320243...,4,0.174533,0.0,0.0,0.5,"[-0.08727804221311625, 0.04866805327869114, -0..."


In [34]:
model_num = 5
b_modes = 3
t_modes = 3

p = rw.base_params(b_modes + t_modes + 1)
sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')

dataset_name = 'JEC2'
# Set JEC Specific parameters
p.rho.value = 1.225
p.s_t.value = 1.345
p.c.value = 0.15
p.EI.value = 61
p.GJ.value = 200
p.rho_t.value = 10
p.e_0.value = 0
p.e_1.value = 0
p.ratio_fwt.value = 0.2

vars_ls =[]
vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
vars_ls.append((p.V,np.linspace(0,40,81))) # V must be second
vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
#vars_ls.append((p.ratio_fwt,[0,0.1,0.2,0.3]))
#vars_ls.append((p.ratio_DL,[0,0.05,0.1,0.2]))
vars_ls.append((p.m_factor,[0.5,1,1.5]))

variables = [k for k,v in vars_ls]

calc_fixed = True

In [35]:
%%time
model_mini = sm.msubs(p.GetSubs(0,p.fp,ignore=variables))
#model_lin = model_mini.linearise(p)

CPU times: user 545 ms, sys: 3.31 ms, total: 548 ms
Wall time: 548 ms


In [36]:
#%%time
#j = model_lin.ExtForces.Q().jacobian(p.qd)

In [37]:
%%time
# get eigen Matrices and turn into a function
K,M = model_mini.GeneralEigenProblemLin(p)


CPU times: user 29.9 s, sys: 28.3 ms, total: 29.9 s
Wall time: 29.9 s


In [38]:
#get free body problem
import sympy.physics.mechanics as me
import sympy as sym
K_v0 = me.msubs(K,{sym.Symbol(p.V.name):0})
M_v0 = me.msubs(M,{sym.Symbol(p.V.name):0})
func = sym.lambdify((variables,p.fp),(K,M))
func_v0 = sym.lambdify((variables,p.fp),(K_v0,M_v0))

In [39]:
from scipy.linalg import eig
perms = np.array(np.meshgrid(*[v for k,v in vars_ls ])).T.reshape(-1,len(vars_ls))
df_perms = [[(vars_ls[i][0],row[i]) for i in range(len(row))] for row in perms]

In [40]:
%%timeit -n 1000 -r 1
values = tuple([v for k,v in df_perms[10]])      
eig(*func_v0(values,[0.01]*14))
eig(*func(values,[0.01]*14))

1.07 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1000 loops each)


In [47]:
%%time
# If caluclating fixed points generate require objective functions
if True:
    f = me.msubs((model_mini.f-model_mini.ExtForces.Q()),{i:0 for i in p.qd})
    func_obj = sym.lambdify((p.q,variables),f)
    func_jac_obj = sym.lambdify((p.q,variables),f.jacobian(p.q),"numpy")

CPU times: user 1min 29s, sys: 89.2 ms, total: 1min 29s
Wall time: 1min 29s


In [62]:
%%time
f = me.msubs((model_mini.f-model_mini.ExtForces.Q()),{i:0 for i in p.qd})
func_obj = sym.lambdify((p.q,variables),f)


#ff =sym.lambdify((p.q,variables),sym.Matrix([[*f.diff(xi)] for xi in p.q]).T,"numpy")

CPU times: user 1min 23s, sys: 72.2 ms, total: 1min 23s
Wall time: 1min 23s


In [46]:
from scipy.optimize import fsolve
# Get all possible combinations of the variables
perms = np.array(np.meshgrid(*[v for k,v in vars_ls ])).T.reshape(-1,len(vars_ls))

df_perms = [[(vars_ls[i][0],row[i]) for i in range(len(row))] for row in perms]
string_perms = [{vars_ls[i][0].name:row[i] for i in range(len(row))} for row in perms]

#Calc freqs and dampings
flutdfv2 = []
qs = []
for i in range(len(df_perms)):
    values = tuple([v for k,v in df_perms[i]])      
    # Calc fixed point
    #set the initial guess (if v=0 set to FWT dropped doen else use previous result)
    if True:
        if string_perms[i]["V"] == 0:
            guess = [0]*p.qs
            guess[-1] = np.pi/2
        elif i == 0:
            guess = [0]*p.qs
        else:
            guess = qs[-1]                      
        q = fsolve(lambda q,v: func_obj(q,values)[:,0],guess,fprime = func_jac_obj ,factor = 1,args=(values,))    
    else:
        q=[0]*p.qs

    qs.append(q)
    x = [0]*p.qs*2
    x[::2] = q
    if string_perms[i]["V"] == 0:
        evals, evecs = eig(*func_v0(values,x))
    else:
        evals, evecs = eig(*func(values,x))

    jac_dat = mf.ExtractEigenValueData_list(evals,evecs,sortby='F')
    #create q and perm data to match size
    for j in range(len(jac_dat)):    
        flutdfv2.append({**jac_dat[j],**string_perms[i],'q':q})

df = pd.DataFrame(flutdfv2)

/Users/fintan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
